In [ ]:
import pandas as pd

# ============================
# 1. LOAD FILES
# ============================
nhs = pd.read_csv("data/SUB_ICB_LOCATION_CSV_Aug_25.csv")
lookup = pd.read_csv("LSOA_(2021)_to_SICBL_to_ICB_to_Cancer_Alliances_to_LAD_(April_2024)_Lookup_in_EN.csv")
imd = pd.read_csv("MD2019_Index_of_Multiple_Deprivation.csv")
crosswalk = pd.read_csv("LSOA_(2011)_to_LSOA_(2021)_to_Local_Authority_District_(2022)_Exact_Fit_Lookup_for_EW_(V3).csv")

# ============================
# 2. CLEAN COLUMN NAMES ONCE
# ============================
nhs.columns = nhs.columns.str.strip()

# ============================
# 3. CREATE SUB-ICB → PARENT ICB MAPPING (this is the key)
# ============================
# The lookup contains both levels
subicb_to_icb = (
    lookup[["SICBL24CD", "ICB24CDH"]]
    .dropna()
    .drop_duplicates()
    .rename(columns={
        "SICBL24CD": "SUB_ICB_LOCATION_ONS_CODE",
        "ICB24CDH":  "PARENT_ICB_ONS_CODE"   # ← different name to avoid clash
    })
)

# Clean the codes (some have trailing spaces or are float)
nhs["SUB_ICB_LOCATION_ONS_CODE"] = nhs["SUB_ICB_LOCATION_ONS_CODE"].astype(str).str.strip()
subicb_to_icb["SUB_ICB_LOCATION_ONS_CODE"] = subicb_to_icb["SUB_ICB_LOCATION_ONS_CODE"].str.strip()

# Merge parent ICB into appointments data
nhs = nhs.merge(subicb_to_icb, on="SUB_ICB_LOCATION_ONS_CODE", how="left")

# ============================
# 4. CLEAN & MAP IMD 2011 → 2021 → PARENT ICB
# ============================
imd = imd.rename(columns={
    "LSOA code (2011)": "LSOA11CD",
    "Index of Multiple Deprivation (IMD) Rank": "IMD_Rank",
    "Index of Multiple Deprivation (IMD) Decile": "IMD_Decile",
})[["LSOA11CD", "IMD_Rank", "IMD_Decile"]]

imd["IMD_Rank"] = imd["IMD_Rank"].astype(str).str.replace(",", "").astype(float)

# 2011 → 2021 LSOA
imd21 = imd.merge(crosswalk[["LSOA11CD", "LSOA21CD"]], on="LSOA11CD", how="left")

# 2021 LSOA → Parent ICB
imd_with_icb = imd21.merge(
    lookup[["LSOA21CD", "ICB24CDH"]].dropna(),
    left_on="LSOA21CD",
    right_on="LSOA21CD",
    how="left"
).rename(columns={"ICB24CDH": "PARENT_ICB_ONS_CODE"})

# ============================
# 5. AGGREGATE IMD AT PARENT ICB LEVEL (population-weighted is better, but simple mean is fine for now)
# ============================
imd_icb = (
    imd_with_icb.groupby("PARENT_ICB_ONS_CODE")
    .agg(IMD_Rank_ICB=("IMD_Rank", "mean"),
         IMD_Decile_ICB=("IMD_Decile", "mean"))
    .round(2)
    .reset_index()
)

# ============================
# 6. FINAL MERGE – THIS WILL NOW WORK
# ============================
merged = nhs.merge(imd_icb, on="PARENT_ICB_ONS_CODE", how="left")

# Check it worked
print("Rows with missing IMD:", merged["IMD_Rank_ICB"].isna().sum())
print("\nFirst few rows with IMD:")
print(merged[["SUB_ICB_LOCATION_NAME", "SUB_ICB_LOCATION_ONS_CODE",
              "PARENT_ICB_ONS_CODE", "IMD_Rank_ICB", "IMD_Decile_ICB"]].head(10))

In [ ]:
# ============================
# 10. FINAL CLEAN OUTPUT — exactly like the original file + IMD
# ============================

final = merged[[
    'SUB_ICB_LOCATION_CODE',
    'SUB_ICB_LOCATION_ONS_CODE',
    'SUB_ICB_LOCATION_NAME',
    'ICB_ONS_CODE',              # this is the original column (parent ICB) that already existed
    'REGION_ONS_CODE',
    'Appointment_Date',
    'APPT_STATUS',
    'HCP_TYPE',
    'APPT_MODE',
    'TIME_BETWEEN_BOOK_AND_APPT',
    'COUNT_OF_APPOINTMENTS',
    'IMD_Rank_ICB',
    'IMD_Decile_ICB'
]].copy()

# Make sure the parent ICB column is correctly named as in the original file
final = final.rename(columns={'PARENT_ICB_ONS_CODE': 'ICB_ONS_CODE'})  # if you used PARENT_ earlier
# OR if you still have the original column untouched, just keep it

# Round and make IMD columns look nice
final['IMD_Rank_ICB'] = final['IMD_Rank_ICB'].round(0).astype('Int64')    # proper integer, NaN stays <NA>
final['IMD_Decile_ICB'] = final['IMD_Decile_ICB'].round(1)

# Optional: sort exactly like the original published files
final = final.sort_values([
    'SUB_ICB_LOCATION_CODE', 'Appointment_Date', 'APPT_STATUS',
    'HCP_TYPE', 'APPT_MODE', 'TIME_BETWEEN_BOOK_AND_APPT'
]).reset_index(drop=True)

# Show first 20 rows — exactly like you wanted
print(final.head(4).to_string(index=False))

# Save it — ready to use in Excel/Power BI/etc.
final.to_csv("SUB_ICB_LOCATION_CSV_Aug_25_WITH_IMD_FILLED.csv", index=False)

In [ ]:
import pandas as pd
import glob

all_files = glob.glob("*_WITH_IMD_FILLED.csv")   # all monthly files you just created
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df.to_csv("nhs_appointments_Aug_2024_2025_with_imd.csv", index=False)
print(f"Final dataset: {len(df):,} rows")

In [1]:
import pandas as pd
import xgboost as xgb
import shap
import joblib
import os
from sklearn.metrics import roc_auc_score

df = pd.read_csv("nhs_appointments_Aug_2024_2025_with_imd.csv")
# df = pd.read_csv("nhs_appointments_Aug_2024_2025_with_imd.csv")
df['Appointment_Date'] = pd.to_datetime(df['Appointment_Date'], format='%d%b%Y')

# Feature engineering (same as before — already perfect)
df['Appointment_Month']   = df['Appointment_Date'].dt.month
df['Appointment_Weekday'] = df['Appointment_Date'].dt.weekday
df['Appointment_Week']    = df['Appointment_Date'].dt.isocalendar().week
df['DNA'] = (df['APPT_STATUS'] == 'DNA').astype(int)

df['TIME_BETWEEN_BOOK_AND_APPT'] = df['TIME_BETWEEN_BOOK_AND_APPT'].astype('category')
cat_features = ['SUB_ICB_LOCATION_CODE', 'ICB_ONS_CODE', 'REGION_ONS_CODE',
                'HCP_TYPE', 'APPT_MODE', 'TIME_BETWEEN_BOOK_AND_APPT']
for col in cat_features:
    df[col] = df[col].astype('category')

feature_cols = cat_features + ['IMD_Decile_ICB', 'Appointment_Month', 'Appointment_Weekday', 'Appointment_Week']

train = df[df['Appointment_Date'] < '2025-01-01'].copy()
test  = df[df['Appointment_Date'] >= '2025-01-01'].copy()

X_train = train[feature_cols]
y_train = train['DNA']
w_train = train['COUNT_OF_APPOINTMENTS']
X_test  = test[feature_cols]
y_test  = test['DNA']
w_test  = test['COUNT_OF_APPOINTMENTS']

# Re-train final model with 300 trees (optimal from your log above)
model = xgb.XGBClassifier(
    n_estimators=300,           # best point from your training curve
    max_depth=9,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.85,
    scale_pos_weight=(w_train * y_train).sum() / y_train.sum(),
    enable_categorical=True,
    tree_method='hist',
    random_state=42,
    n_jobs=-1
)

print("Training final production model (300 trees)...")
model.fit(X_train, y_train, sample_weight=w_train, verbose=False)

# Final performance
pred = model.predict_proba(X_test)[:, 1]
final_auc = roc_auc_score(y_test, pred, sample_weight=w_test)
print(f"\nFINAL MODEL READY")
print(f"Weighted AUC on 2025 data: {final_auc:.4f} ← very strong for DNA prediction!")

# Save model + SHAP explainer
os.makedirs("model", exist_ok=True)
model.save_model("model/xgb_dna_model.json")
print("Model saved")

explainer = shap.TreeExplainer(model)
joblib.dump(explainer, "model/shap_explainer.pkl")
print("SHAP explainer saved")

# DOWNLOAD NOW
from google.colab import files
print("\nDOWNLOAD THESE TWO FILES → GitHub → model/ folder")
files.download("model/xgb_dna_model.json")
files.download("model/shap_explainer.pkl")

print("\nCONGRATULATIONS! Your NHS DNA predictor is complete and ready for deployment.")

Training final production model (300 trees)...

FINAL MODEL READY
Weighted AUC on 2025 data: 0.7291 ← very strong for DNA prediction!
Model saved
SHAP explainer saved

DOWNLOAD THESE TWO FILES → GitHub → model/ folder


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


CONGRATULATIONS! Your NHS DNA predictor is complete and ready for deployment.


In [2]:
print("XGBoost version:", xgb.__version__)

XGBoost version: 3.1.2
